In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

np.set_printoptions(threshold=10, precision=4, suppress=True)
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [ ]:
data.head()

In [ ]:
data = np.array(data)
m,n = data.shape
np.random.shuffle(data)
print(data, 'hello')

data_tt = data.T
print(data_train)
Y_train = data_tt[0]
x_train = data_tt[1:n]
x_train = x_train / 255.

_,m_train = x_train.shape


In [ ]:
def base_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10,1) - 0.5
    return(W1,b1,W2,b2)

def ReLU(Z):
    return np.maximum(Z, 0)

def ReLU_deriv(Z):
    return Z > 0

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_propagation(W1,b1,W2,b2,X):
    Z1 = W1.dot(X) + b1
    a1 = ReLU(Z1)
    Z2 = W2.dot(a1) + b2
    a2 = softmax(Z2)
    return Z1, a1, Z2, a2

def OneHotEncoded(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_propagation(Z1,a1,W1, Z2,a2,W2, X, Y):
    enc_Y = OneHotEncoded(Y)
    dZ2 = a2 - enc_Y
    dW2 = 1 / m * dZ2.dot(a1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1    
    W2 = W2 - learning_rate * dW2  
    b2 = b2 - learning_rate * db2    
    return W1, b1, W2, b2

In [ ]:
def get_predictions(a2):
    return np.argmax(a2,0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y)/ Y.size

def gradient_descent(X, Y, iterations, learning_rate):
    W1,b1,W2,b2 = base_params()
    for i in range(iterations):
        Z1, a1, Z2, a2 = forward_propagation(W1,b1,W2,b2, X)
        dW1, db1, dW2, db2 = backward_propagation(Z1,a1,W1, Z2,a2,W2, X , Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
        if i % 25 == 0:
            print('Iteration: ', i)
            predictions = get_predictions(a2)
            print(get_accuracy(predictions,Y))
    return W1,b1,W2,b2

In [ ]:
W1,b1,W2,b2 = gradient_descent(x_train, Y_train, 500, 0.2)

Checking results

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, a2 = forward_propagation(W1, b1, W2, b2, X)
    predictions = get_predictions(a2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = x_train[:, index, None]
    prediction = make_predictions(x_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
for i in range(10):
    test_prediction(i*4, W1, b1, W2, b2)
    test_prediction(i*1, W1, b1, W2, b2)
    test_prediction(i*2, W1, b1, W2, b2)
    test_prediction(i*3, W1, b1, W2, b2)